# Feature Engineering with SHAP values Experiment 1

SHAP Images of server with different number of poisoned clients
* number of malicious clients [0,1,2,3,4,5]

SHAP Images right after poisoning attack
* number of malicious clients [1,2,3,4,5]

Summation of Differences SHAP Images right after poisoning attack
* rounds [1,2,10,75,200]
* 5 times

## Google Colab

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
sys.path.append('/content/drive/My Drive/Colab Notebooks/federated_learning')

In [ ]:
!pip install shap==0.40.0

## Experiments

In [1]:
from federated_learning.utils import SHAPUtil, experiment_util, Visualizer
from federated_learning import ClientPlane, Configuration, ObserverConfiguration
from federated_learning.server import Server
from datetime import datetime

## MNIST
(1) 5 → 4,
(2) 1 → 7,
(3) 3 → 8,

In [2]:
from federated_learning.nets import MNISTCNN
from federated_learning.dataset import MNISTDataset
import os
config = Configuration()
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = MNISTDataset
config.MODELNAME = config.MNIST_NAME
config.NETWORK = MNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.datasetObserverConfiguration = "MNIST"
neutral_label = 2

In [ ]:
# Google Colab Settigns
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"

In [3]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)
visualizer = Visualizer(shap_util)

MNIST training data loaded.
MNIST test data loaded.
Create 200 clients with dataset of size 300


In [ ]:
import numpy as np
import copy
for i in range(199):
    experiment_util.set_rounds(client_plane, server, i+1)
    experiment_util.run_round(client_plane, server, i+1)
print("Run 199 finished")
old_params = copy.deepcopy(server.get_nn_parameters())

In [ ]:

#import torch
#torch.save(server.net.state_dict(), "temp/models/MNISTtrained2.model")

### alpha (5,4)

In [ ]:
config.FROM_LABEL = 5
config.TO_LABEL = 4

accuracies = []
recalls = []

import torch
torch.save(server.net.state_dict(), "temp/models/MNISTtrained.model")

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    server.update_nn_parameters(old_params)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    file_name = './results/ex3/MNIST/5_4/_run_shap_values_{}_poisoned_clients_alpha_5_4.pdf'.format(j)
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracies.append(accuracy)
    recalls.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

In [ ]:
# Run 0
print(accuracies)
print(recalls)

In [ ]:
# Run 1
print(accuracies[6:])
print(recalls[6:])

In [ ]:
# Run 2
print(accuracies[12:])
print(recalls[12:])

In [ ]:
# Run 3
print(accuracies)
print(recalls)

In [ ]:
# Run 4
print(accuracies)
print(recalls)

### alpha (1,7)

In [ ]:
import copy
import torch
from federated_learning.nets import MNISTCNN
config.FROM_LABEL = 1
config.TO_LABEL = 7

server.net =  MNISTCNN()
server.net.load_state_dict(torch.load('temp/models/MNISTtrained2.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

In [ ]:
accuracies = []
recalls = []

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    server.update_nn_parameters(old_params)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    file_name = './results/ex3/MNIST/1_7/4_run_shap_values_{}_poisoned_clients_alpha_1_7.pdf'.format(j)
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracies.append(accuracy)
    recalls.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

In [ ]:
# Run 0
print(accuracies)
print(recalls)

In [ ]:
# Run 1
print(accuracies[6:])
print(recalls[6:])

In [ ]:
# Run 2
print(accuracies)
print(recalls)

In [ ]:
# Run 3
print(accuracies[6:])
print(recalls[6:])

In [ ]:
# Run 4
print(accuracies)
print(recalls)

### alpha (3,8)

In [4]:
import copy
import torch
from federated_learning.nets import MNISTCNN
config.FROM_LABEL = 3
config.TO_LABEL = 8

server.net =  MNISTCNN()
server.net.load_state_dict(torch.load('temp/models/MNISTtrained2.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

In [14]:
accuracies = []
recalls = []

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    server.update_nn_parameters(old_params)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    file_name = './results/ex3/MNIST/3_8/2_run_shap_values_{}_poisoned_clients_alpha_3_8.pdf'.format(j)
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracies.append(accuracy)
    recalls.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))


Test set: Average loss: 0.0001, Accuracy: 9749/10000 (97%)

Original tensor([0.9929, 0.9921, 0.9787, 0.9832, 0.9745, 0.9709, 0.9823, 0.9582, 0.9548,
        0.9594]) tensor([0.9653, 0.9869, 0.9555, 0.9585, 0.9846, 0.9807, 0.9864, 0.9733, 0.9862,
        0.9738]) 0.9749
No poisoning due to 0. poisoned clients
[110, 22, 170, 185, 52]
Train Epoch: 200 [0/300 (0%)]	Loss: 0.107917
Train Epoch: 200 [10/300 (3%)]	Loss: 0.092170
Train Epoch: 200 [20/300 (7%)]	Loss: 0.223941
Train Epoch: 200 [30/300 (10%)]	Loss: 0.612377
Train Epoch: 200 [40/300 (13%)]	Loss: 0.029483
Train Epoch: 200 [50/300 (17%)]	Loss: 0.023221
Train Epoch: 200 [60/300 (20%)]	Loss: 0.111617
Train Epoch: 200 [70/300 (23%)]	Loss: 0.040513
Train Epoch: 200 [80/300 (27%)]	Loss: 0.454605
Train Epoch: 200 [90/300 (30%)]	Loss: 0.073538
Train Epoch: 200 [100/300 (33%)]	Loss: 0.292561
Train Epoch: 200 [110/300 (37%)]	Loss: 0.009152
Train Epoch: 200 [120/300 (40%)]	Loss: 0.278231
Train Epoch: 200 [130/300 (43%)]	Loss: 0.121038
Train E

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.


make picture
Make picture
tensor([0.9929, 0.9903, 0.9816, 0.9168, 0.9796, 0.9798, 0.9833, 0.9611, 0.9702,
        0.9653]) tensor([0.9701, 0.9877, 0.9602, 0.9957, 0.9816, 0.9552, 0.9812, 0.9734, 0.9459,
        0.9692]) 0.9721
Poisoned clients: 0

Test set: Average loss: 0.0001, Accuracy: 9749/10000 (97%)

Original tensor([0.9929, 0.9921, 0.9787, 0.9832, 0.9745, 0.9709, 0.9823, 0.9582, 0.9548,
        0.9594]) tensor([0.9653, 0.9869, 0.9555, 0.9585, 0.9846, 0.9807, 0.9864, 0.9733, 0.9862,
        0.9738]) 0.9749
Poison 1/200 clients
Flip 100.0% of the 3 labels to 8
[134]
[72, 93, 151, 79, 134]
Train Epoch: 200 [0/300 (0%)]	Loss: 0.037225
Train Epoch: 200 [10/300 (3%)]	Loss: 0.009104
Train Epoch: 200 [20/300 (7%)]	Loss: 0.909985
Train Epoch: 200 [30/300 (10%)]	Loss: 0.759963
Train Epoch: 200 [40/300 (13%)]	Loss: 0.032215
Train Epoch: 200 [50/300 (17%)]	Loss: 1.172093
Train Epoch: 200 [60/300 (20%)]	Loss: 0.643163
Train Epoch: 200 [70/300 (23%)]	Loss: 0.483580
Train Epoch: 200 [80/300 (2

In [10]:
# Run 0
print(accuracies)
print(recalls)

[0.9764, 0.9754, 0.9727, 0.9529, 0.8817, 0.8767]
[tensor([0.9939, 0.9938, 0.9738, 0.9812, 0.9766, 0.9731, 0.9770, 0.9669, 0.9610,
        0.9643]), tensor([0.9929, 0.9903, 0.9806, 0.9693, 0.9633, 0.9798, 0.9823, 0.9523, 0.9733,
        0.9693]), tensor([0.9929, 0.9903, 0.9758, 0.9218, 0.9776, 0.9854, 0.9823, 0.9640, 0.9723,
        0.9653]), tensor([0.9949, 0.9912, 0.9671, 0.7287, 0.9817, 0.9843, 0.9812, 0.9737, 0.9661,
        0.9623]), tensor([0.9929, 0.9921, 0.9826, 0.0307, 0.9776, 0.9765, 0.9812, 0.9562, 0.9702,
        0.9653]), tensor([0.9929, 0.9930, 0.9641, 0.0000, 0.9786, 0.9630, 0.9833, 0.9650, 0.9743,
        0.9604])]


In [13]:
# Run 1
print(accuracies)
print(recalls)

[0.9761, 0.9562, 0.9728, 0.9531, 0.8871, 0.8793]
[tensor([0.9929, 0.9894, 0.9748, 0.9673, 0.9735, 0.9809, 0.9791, 0.9689, 0.9702,
        0.9633]), tensor([0.9929, 0.9921, 0.9758, 0.7653, 0.9786, 0.9787, 0.9854, 0.9660, 0.9702,
        0.9584]), tensor([0.9929, 0.9903, 0.9816, 0.9168, 0.9695, 0.9843, 0.9833, 0.9650, 0.9754,
        0.9693]), tensor([0.9918, 0.9921, 0.9758, 0.7366, 0.9766, 0.9798, 0.9854, 0.9640, 0.9723,
        0.9584]), tensor([0.9918, 0.9885, 0.9729, 0.0871, 0.9735, 0.9787, 0.9854, 0.9640, 0.9733,
        0.9643]), tensor([0.9918, 0.9894, 0.9758, 0.0000, 0.9837, 0.9865, 0.9760, 0.9611, 0.9764,
        0.9623])]


In [ ]:
# Run 2
print(accuracies)
print(recalls)

In [ ]:
# Run 3
print(accuracies)
print(recalls)

In [ ]:
# Run 4
print(accuracies)
print(recalls)

### FashionMNIST
For Fashion-MNIST we experiment with 
(1) 5: sandal → 4: coat,
(2) 2: pullover → 3: shirt, and 
(3) 4: coat → 6: dress.
['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker',  'Bag', 'Ankle Boot']
